In [1]:
pip install rapidfuzz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sentence_transformers import SentenceTransformer
from torch.nn.functional import cosine_similarity
import torch
from rapidfuzz import fuzz

In [3]:
titles = []
summaries = []

with open("books.txt", "r", encoding="utf-8") as f:
    for line in f:
        if " - " in line:
            title, summary = line.strip().split(" - ", 1)
            titles.append(title)
            summaries.append(summary)

In [4]:
model = SentenceTransformer('all-mpnet-base-v2') 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\alpan\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alpan\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
summary_embeddings = model.encode(summaries, convert_to_tensor=True)

In [22]:
query = input("Enter a quote or sentence from the book: ")
query_embedding = model.encode(query, convert_to_tensor=True)

Enter a quote or sentence from the book:  wizard


In [23]:
# Semantic similarity
semantic_scores = cosine_similarity(query_embedding, summary_embeddings).tolist()

# Fuzzy similarity
fuzzy_scores = [fuzz.partial_ratio(query, summary) / 100 for summary in summaries]

In [24]:
final_scores = [
    0.7 * semantic + 0.3 * fuzzy
    for semantic, fuzzy in zip(semantic_scores, fuzzy_scores)
]
best_idx = final_scores.index(max(final_scores))
print(f"\n Best Matched Book:\n{titles[best_idx]} (Combined Similarity Score: {final_scores[best_idx]:.4f})")


 Best Matched Book:
Harry Potter and the Sorcerer's Stone (Combined Similarity Score: 0.5979)
